In [1]:
import pandas as pd
import feather

from keras.models import Sequential
from keras.layers import Dense
from keras.utils.np_utils import to_categorical
from keras.models import load_model


Using TensorFlow backend.


In [2]:
n_groups_per_col = 9
architecture = "2_dense_512"
spread = "spread"
path = "../data/models/ind/binary/" + architecture +"/" + spread + "/"

data_train = pd.read_feather(path + "data_train.feather")

labels_low_train = pd.read_feather(path + "labels_low_train.feather")
labels_high_train = pd.read_feather(path + "labels_high_train.feather")
labels_close_train = pd.read_feather(path + "labels_close_train.feather")

In [3]:
train_data = data_train.to_numpy()
labels_low_train = labels_low_train.to_numpy()
labels_high_train = labels_high_train.to_numpy()
labels_close_train = labels_close_train.to_numpy()

In [4]:
def fit_ind_binary_model(data_train, labels_train):
    
    model = Sequential()
    model.add(Dense(units=512, activation='relu', input_shape=(data_train.shape[1],)))
    model.add(Dense(units=512, activation='relu'))
    model.add(Dense(labels_train.shape[1], activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    history = model.fit(data_train, labels_train, validation_split=0.2, batch_size=256, epochs=3, verbose=1)
    
    return (history, model)

In [ ]:
# low
(low_history, low_model) = fit_ind_binary_model(data_train, labels_low_train)
feather.write_dataframe(pd.DataFrame.from_dict(low_history.history), path + "low_history.feather", compression = "uncompressed")
low_model.save(path + "low_model.hdf5")


In [ ]:
# high
(high_history, high_model) = fit_ind_binary_model(train_data, labels_high_train)
feather.write_dataframe(pd.DataFrame.from_dict(high_history.history), path + "high_history.feather", compression = "uncompressed")
high_model.save(path + "high_model.hdf5")

In [5]:
# close
(close_history, close_model) = fit_ind_binary_model(train_data, labels_close_train)
feather.write_dataframe(pd.DataFrame.from_dict(close_history.history), path + "history.feather", compression = "uncompressed")
close_model.save(path + "close_model.hdf5")


Train on 3566135 samples, validate on 891534 samples
Epoch 1/3
3566135/3566135 [==============================] - 49s 14us/step - loss: 0.4949 - accuracy: 0.7514 - val_loss: 0.4866 - val_accuracy: 0.7530
Epoch 2/3
3566135/3566135 [==============================] - 49s 14us/step - loss: 0.4851 - accuracy: 0.7531 - val_loss: 0.4890 - val_accuracy: 0.7530
Epoch 3/3
3566135/3566135 [==============================] - 48s 14us/step - loss: 0.4845 - accuracy: 0.7531 - val_loss: 0.4851 - val_accuracy: 0.7530


In [7]:
### Predict

low_model = load_model(path + "low_model.hdf5")
high_model = load_model(path + "high_model.hdf5")
# close_model = load_model("../data/models/ind/binary/close_model.hdf5")

In [9]:
data_test

array([[109.27324338, 109.27324358, 106.26570126, ..., 107.26814527,
        104.76189631, 100.        ],
       [ 98.52213901, 100.00001182,  98.52217435, ..., 102.95561555,
        100.        , 100.        ],
       [ 98.12200104,  95.7746988 ,  97.65257239, ..., 104.22531046,
        102.81681004, 100.        ],
       ...,
       [ 96.01658106,  97.25995986,  97.2829868 , ..., 100.87497539,
        101.28943269, 100.        ],
       [102.22598893, 101.91145033, 104.28260646, ..., 105.08105507,
        104.59714506, 100.        ],
       [111.5778721 , 111.68032959, 112.37193103, ..., 103.84221317,
        104.17520959, 100.        ]])

In [8]:
data_test = pd.read_feather(path + "data_test.feather").to_numpy()

low_pred_prob = low_model.predict_proba(data_test, batch_size = 256)
feather.write_dataframe(pd.DataFrame(low_pred_prob), path + "low_pred_prob.feather", compression='uncompressed')

high_pred_prob = high_model.predict_proba(data_test, batch_size = 256)
feather.write_dataframe(pd.DataFrame(high_pred_prob), path + "high_pred_prob.feather", compression='uncompressed')

close_pred_prob = close_model.predict_proba(data_test, batch_size = 256)
feather.write_dataframe(pd.DataFrame(close_pred_prob), path + "close_pred_prob.feather", compression='uncompressed')
